In [1]:
import pandapower.networks as pn
import pandas as pd
from pandas import DataFrame
import pandapower as pp #import pandapower
import pandapower.topology as top
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import sklearn
%matplotlib inline

In [2]:
net = pp.create_empty_network(f_hz=60.0) #create an empty network
#Create buses
bus1 = pp.create_bus(net, vn_kv=20.5, name = "bus1")
bus2 = pp.create_bus(net, vn_kv=20.5, name = "bus2")

#create 20.5 kV lines
line1=pp.create_line_from_parameters(net, from_bus = bus1, to_bus = bus2, length_km=20, r_ohm_per_km = .121, x_ohm_per_km = 0.107, c_nf_per_km = 0, max_i_ka = 0.4)

#create loads
pp.create_load(net, bus1, p_mw=20, q_mvar=5, controllable=False)
pp.create_load(net, bus1, p_mw=30, q_mvar=15, controllable=False)
#pp.create_load(net, bus1, p_mw=10, q_mvar=5, controllable=False)
#pp.create_load(net, bus1, p_mw=10, q_mvar=5, controllable=False)

pp.create_load(net, bus2, p_mw=40, q_mvar=15, controllable=False)
#pp.create_load(net, bus2, p_mw=20, q_mvar=5, controllable=False)
#pp.create_load(net, bus2, p_mw=20, q_mvar=5, controllable=False)

#pp.create_load(net, bus3, p_mw=5, q_mvar=5, controllable=False)

#Create External Grid
eg = pp.create_ext_grid(net, bus2, min_p_mw=-1000, max_p_mw=1000)

#Create renewables (Wind park)
pp.create_sgen(net, bus1, p_mw=20, q_mvar=5, sn_mva=100, type='WP', name='Wind Park')
pp.create_sgen(net, bus1, p_mw=20, q_mvar=10, sn_mva=100, type='WP', name='PV')
pp.create_sgen(net, bus2, p_mw=20, q_mvar=10, sn_mva=100, type='WP', name='PV')


#pp.create_sgen(net, bus2, p_mw=20, q_mvar=15, sn_mva=100, type='WP', name='Wind Park')




2

In [3]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type,controllable
0,None,0,20.0,5.0,0.0,0.0,NaN,1.0,True,None,False
1,None,0,30.0,15.0,0.0,0.0,NaN,1.0,True,None,False
2,None,1,40.0,15.0,0.0,0.0,NaN,1.0,True,None,False


In [4]:
net.load.p_mw[0] = net.load.p_mw[0]*1.1;
net.load.q_mvar[0] = net.load.q_mvar[0]*1.1;
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type,controllable
0,None,0,22.0,5.5,0.0,0.0,NaN,1.0,True,None,False
1,None,0,30.0,15.0,0.0,0.0,NaN,1.0,True,None,False
2,None,1,40.0,15.0,0.0,0.0,NaN,1.0,True,None,False


In [9]:
##Change the load using random
def inputData(mu, sigma, n):
    temp1 = net.load.p_mw[0]
    temp2 = net.load.q_mvar[0]
    if n>=499 :
        net.load.p_mw[0] = temp1*1.05;
        net.load.q_mvar[0] = temp2*0.95
    noise = np.random.normal(mu, sigma, [3,2]) 
    net.load.p_mw += noise[:,0]
    net.load.q_mvar += noise[:,1]
    
    
    #Powerflow
    pp.runpp(net)

    V=(DataFrame(net.res_bus, columns = ['vm_pu','va_degree']))
    Z=(DataFrame(net.line, columns = ['length_km','r_ohm_per_km','x_ohm_per_km']))
    L = DataFrame(net.load, columns = ['p_mw','q_mvar'])
    L = -L
    DER = net.res_sgen

    T1=(L.iloc[0:2])
    T1=T1.append(DER.iloc[0 : 2])

    T2 = L.iloc[2:]
    T2=T2.append(DER.iloc[2 : ]).append(net.res_ext_grid) 
    net.load.p_mw[0] = temp1
    net.load.q_mvar[0] = temp2
    return T1,T2,V,Z

In [10]:
temp=inputData(0,0.1,0)
T1 = temp[0]
T2 = temp[1]
V= temp[2]
Z = temp[3]
for i in range(999):
    temp=inputData(0,0.1,i)
    T1=T1.append(temp[0])
    T2=T2.append(temp[1])
    V=V.append(temp[2])
    Z=Z.append(temp[3])
V.to_csv (r'C:\Users\LEEPS\Desktop\electric_dissection\EnergiesPaperCsv\V.csv', index = None, header=False)
Z.to_csv (r'C:\Users\LEEPS\Desktop\electric_dissection\EnergiesPaperCsv\Z.csv', index = None, header=False)
T1.to_csv (r'C:\Users\LEEPS\Desktop\electric_dissection\EnergiesPaperCsv\T1.csv', index = None, header=False)
T2.to_csv (r'C:\Users\LEEPS\Desktop\electric_dissection\EnergiesPaperCsv\T2.csv', index = None, header=False)


In [7]:
    pp.runpp(net)

    V=(DataFrame(net.res_bus, columns = ['vm_pu','va_degree']))
    Z=(DataFrame(net.line, columns = ['length_km','r_ohm_per_km','x_ohm_per_km']))
    L = DataFrame(net.load, columns = ['p_mw','q_mvar'])
    L = -L
    DER = net.res_sgen

    T1=(L.iloc[0:2])
    T1=T1.append(DER.iloc[0 : 2])

    T2 = L.iloc[2:]
    T2=T2.append(DER.iloc[2 : ]).append(net.res_ext_grid) 
V.to_csv (r'C:\Users\LEEPS\Desktop\electric_dissection\EnergiesPaperCsv\V.csv', index = None, header=False)
Z.to_csv (r'C:\Users\LEEPS\Desktop\electric_dissection\EnergiesPaperCsv\Z.csv', index = None, header=False)
T1.to_csv (r'C:\Users\LEEPS\Desktop\electric_dissection\EnergiesPaperCsv\T1.csv', index = None, header=False)
T2.to_csv (r'C:\Users\LEEPS\Desktop\electric_dissection\EnergiesPaperCsv\T2.csv', index = None, header=False)
